# TD5 : génération de texte avec un LSTM

Ce notebook fait partie du cours sur le deep learning donné par J. Velcin et J. Cugliari à l'Université de Lyon 2 (Master Data Mining). Il reprend et détaille l'exemple donné dans le livre "Deep Learning with Keras" de A. Gulli et S. Pal (Packt Publishing Ltd., 2017).

On construit tout d'abord une unique chaîne de caractères qui provient de la concaténation du texte trouvé dans un ou plusieurs fichiers txt.

In [1]:
import os

lines=[]

for filename in ["rois.txt"]:
     with open(os.path.join("datasets", filename)) as f:    
        for r in f.readlines():
            line = r.strip().lower()
            if (len(line) == 0):
                continue
            lines.append(line)
text = " ".join(lines)

In [2]:
#text = text[0:100000]
#len(text)

On construit un dictionnaire basé sur les caractères trouvés dans les textes car ceux-ci constituent notre *vocabulaire* puisque la génération sera réalisée caractère par caractère.

On note qu'on construit un dictionnaire à double entrée.

In [3]:
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [4]:
#chars

L'étape suivante consiste à construire les exemples d'apprentissage, à savoir :
    - un ensemble de chaînes de taille fixe (par ex. 10)
    - l'étiquette à prédire dans "chars", à savoir le caractère suivant
Pour cela, on fait passer une fenêtre glissante le long du texte avec :
$$i_1 = (c_{i-10}, c_{i-9}\ldots c_{i-1})$$
$$o_1 = c_i$$
si $c_i$ est le caractère à prédire.

In [5]:
len_seq = 10
step = 1
input_chars = []
label_chars = []
for i in range(0, len(text) - len_seq, step):
    input_chars.append(text[i:i + len_seq])
    label_chars.append(text[i + len_seq])

Cela revient à estimer un modèle de langue tel que :
    $$p(c_i / c_{i-10}, c_{i-9}\ldots c_{i-1})$$
où p(c_i) prend une valeur parmi l'ensemble "chars"

In [6]:
print("Il y a ", len(input_chars), " exemples d'apprentissage !")

Il y a  534447  exemples d'apprentissage !


Ensuite on construit les tenseurs à la main :
- X[i,j,k] : observation i, temps j, feature k
- y[i,k] : observation i, feature k

In [7]:
import numpy as np
X = np.zeros((len(input_chars), len_seq, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

Puis on construit le RNN comme dans l'exemple précédent, la différence étant que la prédiction est sur n classes, d'où l'utilisation de la couche softmax.

In [8]:
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(len_seq, nb_chars)))
#model.add(LSTM(10, return_sequences=False))
#model.add(Dropout(0.5))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               69200     
_________________________________________________________________
dense (Dense)                (None, 72)                7272      
_________________________________________________________________
activation (Activation)      (None, 72)                0         
Total params: 76,472
Trainable params: 76,472
Non-trainable params: 0
_________________________________________________________________
None


L'optimiseur "rmsprop" est plus efficace sur l'estimation des paramètres d'un RNN.

Il est temps de lancer l'apprentissage proprement dit, mais en mettant en place un monitoring afin d'observer les résultat de l'apprentissage à chaque pas de temps.

Pour cela, l'idée consiste à prendre un échantillon de 10 caractères au hasard et d'utiliser le modèle de langue sur une longueur de 100 caractères en prenant à chaque fois le caractère qui maximise la probabilité (*argmax*). Il serait possible de faire un tirage aléatoire à partir de p.

In [ ]:
for iteration in range(100):
    print("="*50)
    print("Iteration #: %d" % (iteration))
    model.fit(X,y, batch_size=128, epochs=1)
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(100):
        Xtest = np.zeros((1, len_seq, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
            #p = np.nonzero(np.random.multinomial(1, pred))[0][0]
            #ypred = index2char[p]
        print(ypred, end="")
        test_chars = test_chars[1:] + ypred
            

Iteration #: 0
4176/4176 [==============================] - 91s 22ms/step - loss: 2.2059
Generating from seed:  quelque p
 quelque parient de le reine de la comme de la chande de le reine de la comme de la chande de le reine de la c==================================================
Iteration #: 1
4176/4176 [==============================] - 116s 28ms/step - loss: 1.8193
Generating from seed: ci », pens
ci », pensait le roi de la comment de la courtait de la courtait de la courtait de la courtait de la courtait ==================================================
Iteration #: 2
4176/4176 [==============================] - 108s 26ms/step - loss: 1.6793
Generating from seed: es victime
es victiment de la complit de la complit de la complit de la complit de la complit de la complit de la complit==================================================
Iteration #: 3
3984/4176 [===========================>..] - ETA: 4s - loss: 1.6015-

mytest = "why are yo"

for i in range(100):
    Xtest = np.zeros((1, len_seq, nb_chars))
    for i, ch in enumerate(mytest):
        Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred, end="")
    mytest = mytest[1:] + ypred
